In [1]:
# Import packages

# NSM Packages
from National_Snow_Model import SWE_Prediction
import National_Snow_Model
import NSM_SCA


# Dataframe Packages
import numpy as np
import xarray as xr
import pandas as pd

# Vector Packages
import geopandas as gpd
import shapely

# Raster Packages
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import rasterstats as rs

# Data Access Packages
import earthaccess as ea
from nsidc_fetch import download, format_date, format_boundingbox
import h5py
import pickle
from tensorflow.keras.models import load_model

# General Packages
import re
from datetime import datetime
import glob
from pprint import pprint
from typing import Union
from pathlib import Path
from tqdm import tqdm
import time
import os
import functools
import operator
from tqdm import tqdm

# Ignore some matplotlib deprecation warnings
import warnings; warnings.simplefilter("ignore")



C:\Users\rjohnson18\Anaconda3\envs\NSM_env\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(
C:\Users\rjohnson18\Anaconda3\envs\NSM_env\lib\site-packages\xarray\backends\cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
# function to get unique values
def unique(list1):
 
    # initialize a null list
    unique_list = []
 
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list


In [3]:
#Set working directories
cwd = os.getcwd()
os.chdir("..")
datapath = os.getcwd()   

## Load and process the training data

The training data needs to be combined into a large DF with all regions present and subsampled by date to connect the VIIRS SCA observations. 
For each region, we select the lat/long and date for each grid cell.

In [4]:
#load in training data and get unique dates
#set data path
training_path = f"{datapath}/Data/VIIRS_SCA/Training_Testing_Data/Provided_Data/"

#get region ids
Regions = pd.read_csv(f"{training_path}/Regions.csv")
Region_list =list(Regions['Regions'].values)

#load training data
TrainDF = pd.DataFrame()
Dates = []
cols = ['Date', 'Long', 'Lat']
bbox = [-124,33,-106,49]

print('Processing training data regions into one dataframe')
for region in tqdm(Region_list):   
    #load the RegionTrain DF, select the key grid cell colums, and add to training DF.
    RegionDF = pd.read_hdf(f"{training_path}/RegionTrain_Final.h5", region)
    frames = [TrainDF, RegionDF[cols]]
    TrainDF = pd.concat(frames)
    
#load dates
Dates = pd.read_csv(f"{training_path}/dates.csv")
Dates = list(Dates['Date'].values)

#Make a DF for each date to augment SCA
DateDict = {}
print('Collecting dates within training data')
for date in tqdm(Dates):
    DateDict[date] = TrainDF[TrainDF['Date'] ==date]
    #add variable columns
    DateDict[date]['VIIRS_SCA'] =0.0
    DateDict[date]['hasSnow'] = 'False'
    

Processing training data regions into one dataframe


100%|██████████| 23/23 [00:00<00:00, 48.58it/s]


100%|██████████| 209/209 [00:00<00:00, 1135.29it/s]


## Connect downloaded VIIRS fSCA daily products to lat/long/datetime of training dataframe

Adapted from the NSM_SCA.py script

In [5]:
print('Connecting the VIIRS SCA observations to lat/long/dates')
for date in tqdm(Dates):
    print(date)
#set up datetime inputs
    #date = '2013-01-08' #datetime(2018, 12, 24)
    date = pd.to_datetime(date, format='%Y-%m-%d' )
    timeDelay = 0
    delayedDate = date - pd.Timedelta(days=timeDelay)
    delayedDatestr = delayedDate.strftime('%Y-%m-%d')

    #set VIIRS data destination
    #Change the working directory
    if delayedDatestr > f"{delayedDate.year}-09-30":
        folder = f"{delayedDate.year}-{delayedDate.year+1}NASA"

    else:
        folder = f"{delayedDate.year-1}-{delayedDate.year}NASA"


    #This is the folder location of the VIIRS data.
    dataFolder = str(f"{datapath}/Data/VIIRS_SCA/{folder}")

    superset = DateDict[delayedDatestr]
    superset = gpd.GeoDataFrame(superset, geometry=gpd.points_from_xy(superset.Long, superset.Lat, crs="EPSG:4326"))
    superset.total_bounds # This box is likely not necessary as we aready have the data downloaded.

    #Connect
    extentDF = NSM_SCA.calculateGranuleExtent(bbox, delayedDate)  # Get granule extent
    granules = NSM_SCA.fetchGranules(bbox, dataFolder, delayedDate, extentDF, False)  # Fetch granules
    raster = NSM_SCA.createMergedRxr(granules["filepath"])
    DateDict[delayedDatestr] = NSM_SCA.augment_SCA_TDF(superset, raster, delayedDate, dataFolder, threshold = 20)

Connecting the VIIRS SCA observations to lat/long/dates


  0%|          | 0/209 [00:00<?, ?it/s]

2013-01-08
VIIRS version:  1


  0%|          | 1/209 [00:02<08:18,  2.40s/it]

2013-01-15
VIIRS version:  1


...

 99%|█████████▉| 207/209 [21:46<00:14,  7.21s/it]

2019-12-24
VIIRS version:  2


100%|█████████▉| 208/209 [21:55<00:07,  7.48s/it]

2019-12-31
VIIRS version:  2


100%|██████████| 209/209 [22:04<00:00,  6.34s/it]


In [6]:
for date in Dates:
    print(date)

2013-01-08
2013-01-15
2013-01-22
2013-01-29
2013-02-05
2013-02-12
2013-02-19
2013-02-26
2013-03-05
2013-03-12
2013-03-19
2013-03-26
2013-04-02
2013-04-09
2013-04-16
2013-04-23
2013-04-30
2013-05-07
2013-05-14
2013-05-21
2013-05-28
2013-06-01
2013-06-04
2013-06-08
2013-06-11
2013-06-18
2013-06-25
2013-12-10
2013-12-17
2013-12-24
2013-12-31
2014-01-07
2014-01-14
2014-01-21
2014-01-28
2014-02-04
2014-02-11
2014-02-18
2014-02-25
2014-03-04
2014-03-11
2014-03-18
2014-03-25
2014-04-01
2014-04-08
2014-04-15
2014-04-22
2014-04-29
2014-05-06
2014-05-13
2014-05-20
2014-05-27
2014-06-03
2014-06-10
2014-06-17
2014-06-24
2014-12-09
2014-12-16
2014-12-23
2014-12-30
2015-01-06
2015-01-13
2015-01-20
2015-01-27
2015-02-03
2015-02-10
2015-02-17
2015-02-24
2015-03-03
2015-03-10
2015-03-17
2015-03-24
2015-03-31
2015-04-07
2015-04-14
2015-04-21
2015-04-28
2015-05-05
2015-05-12
2015-05-19
2015-05-26
2015-06-02
2015-06-09
2015-06-16
2015-06-23
2015-06-30
2015-12-08
2015-12-15
2015-12-22
2015-12-29
2016-01-05

## Put DataFrames back together and connect with regions

In [7]:
TrainDF_SCA = pd.DataFrame()
for date in Dates:
    TrainDF_SCA = pd.concat([TrainDF_SCA, DateDict[date]])

In [8]:
#Pass the DF through the region_id function from the NSM
TrainDF_SCA['Region'] = 'N/A'
TrainDF_SCA = NSM_SCA.Region_id(TrainDF_SCA)

100%|██████████| 45373/45373 [00:12<00:00, 3597.10it/s]


In [9]:
RegionTrain_SCA = {}

#Put into RegionTrain DataFrame
for region in tqdm(Region_list):   
    #select the data for each region
    Region_SCA = TrainDF_SCA[TrainDF_SCA['Region'] == region]
    #load the RegionTrain DF, select the key grid cell colums, and add to training DF.
    Region_DF = pd.read_hdf(f"{training_path}/RegionTrain_Final.h5", region)

    #select key columns
    cols = ['Date', 'Long', 'Lat', 'VIIRS_SCA', 'hasSnow']
    Region_SCA =Region_SCA[cols]
    
    #reset indices
    Region_SCA.reset_index(inplace = True)
    Region_DF.reset_index(inplace = True)

    #Merge DFs by lat/long/date
    Merged = pd.merge(Region_DF, Region_SCA,  how='left', left_on=['index', 'Date','Long', 'Lat'], right_on = ['index','Date','Long', 'Lat'])
    Merged.set_index('index', inplace = True, drop = True)
    
    #Put into RegionTrainDF
    RegionTrain_SCA[region] = Merged
    
    #Save to h5
    RegionTrain_SCA[region].to_hdf(f"{datapath}/Data/VIIRS_SCA/Training_Testing_Data/Provided_Data/RegionTrain_SCA.h5", region)


100%|██████████| 23/23 [00:01<00:00, 16.56it/s]
